In [1]:
# load packages
import pandas as pd
import numpy as np
import glob
import os
from datetime import datetime

In [2]:
# get path
path = os.getcwd()+"/Data"

In [3]:
# get all files in the folder
all_csv = glob.glob(path+"/*.csv")

In [4]:
# loop through all_csv and concatinate them into one big csv
bike_data = (pd.read_csv(file) for file in all_csv)
bike_data_df = pd.concat(bike_data,ignore_index= True) 

In [5]:
bike_data_df.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')

In [6]:
# check shape
bike_data_df.shape

(608029, 13)

In [7]:
# Data files in Jan and Feb have different column names than the rest of the files, these months will not be included in the analysis.

In [8]:
# start cleaning data
#check for any blanks
bike_data_df[bike_data_df == ""].sum()

ride_id                 0
rideable_type           0
started_at              0
ended_at                0
start_station_name      0
start_station_id        0
end_station_name        0
end_station_id          0
start_lat             0.0
start_lng             0.0
end_lat               0.0
end_lng               0.0
member_casual           0
dtype: object

In [9]:
# check for any NA values
bike_data_df.isna().sum()

ride_id                  0
rideable_type            0
started_at               0
ended_at                 0
start_station_name       0
start_station_id         0
end_station_name      3065
end_station_id        3065
start_lat                0
start_lng                0
end_lat               2010
end_lng               2010
member_casual            0
dtype: int64

In [10]:
missing_percent = bike_data_df[bike_data_df.isna().any(1)].count().max()/bike_data_df.shape[0]

print(f"{round(missing_percent*100,2)}% of the data is missing")

0.5% of the data is missing


In [11]:
#So all the NA values will be removed

In [12]:
bike_data_df.dropna(inplace=True)

In [13]:
bike_data_df.isna().sum()

ride_id               0
rideable_type         0
started_at            0
ended_at              0
start_station_name    0
start_station_id      0
end_station_name      0
end_station_id        0
start_lat             0
start_lng             0
end_lat               0
end_lng               0
member_casual         0
dtype: int64

In [14]:
bike_data_df.shape

(604964, 13)

In [15]:
#Split start_date into start_year/month/day/hour and end_date into end_year/month/day/hour and start_date/end_date

In [16]:
bike_data_df[["Start_Year","Start_Month","Start_Day"]]=bike_data_df["started_at"].str.split("-",expand = True)

In [17]:
bike_data_df[["Start_Day","Start_Time"]] = bike_data_df["Start_Day"].str.split(" ",expand=True)

In [18]:
bike_data_df[["Start_Hour","Start_Minute","Start_Second"]] = bike_data_df["Start_Time"].str.split(":",expand=True)

In [19]:
bike_data_df[["End_Year","End_Month","End_Day"]]=bike_data_df["ended_at"].str.split("-",expand = True)

In [20]:
bike_data_df[["End_Day","End_Time"]] = bike_data_df["End_Day"].str.split(" ",expand=True)

In [21]:
bike_data_df[["End_Hour","End_Minute","End_Second"]] = bike_data_df["End_Time"].str.split(":",expand=True)

In [22]:
bike_data_df["Start_date"] = bike_data_df["Start_Year"].str.cat(bike_data_df["Start_Month"].str.cat(bike_data_df["Start_Day"],sep="/"),sep="/")

In [23]:
bike_data_df["End_date"] = bike_data_df["End_Year"].str.cat(bike_data_df["End_Month"].str.cat(bike_data_df["End_Day"],sep="/"),sep="/")

In [24]:
# add ride duration column

In [25]:
start = pd.to_datetime(bike_data_df["started_at"],format = "%Y-%m-%d %H:%M:%S")
end = pd.to_datetime(bike_data_df["ended_at"],format = "%Y-%m-%d %H:%M:%S")
end

0        2021-07-31 14:47:00
1        2021-07-17 19:20:07
2        2021-07-08 20:43:06
3        2021-07-18 20:28:23
4        2021-07-08 18:24:48
                 ...        
608024   2021-03-29 19:38:10
608025   2021-03-12 18:54:35
608026   2021-03-28 12:34:40
608027   2021-03-23 13:10:25
608028   2021-03-24 10:21:17
Name: ended_at, Length: 604964, dtype: datetime64[ns]

In [26]:
bike_data_df["Ride Duration"] = end-start

In [27]:
bike_data_df["Ride Duration Seconds"] = round(bike_data_df["Ride Duration"]/pd.Timedelta(seconds = 1))

In [28]:
bike_data_df["Ride Duration Mins"]= round(bike_data_df["Ride Duration"]/pd.Timedelta(minutes = 1))

In [29]:
bike_data_df["Ride Duration Hours"] = round(bike_data_df["Ride Duration"]/pd.Timedelta(hours = 1),2)

In [30]:
bike_data_df["Ride Duration Days"] = round(bike_data_df["Ride Duration"]/pd.Timedelta(days = 1),2)

In [31]:
# remove duration less than 0, as duration can not be negative
bike_data_df = bike_data_df[bike_data_df["Ride Duration Seconds"] > 0]

In [32]:
# remove duration (day) more than 1, as it is assumed people do not ride for more than 1 whole day

bike_data_df = bike_data_df[bike_data_df["Ride Duration Days"] < 1]

In [33]:
bike_data_df.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,End_Hour,End_Minute,End_Second,Start_date,End_date,Ride Duration,Ride Duration Seconds,Ride Duration Mins,Ride Duration Hours,Ride Duration Days
0,2F01C15ECE379AE7,classic_bike,2021-07-31 13:53:26,2021-07-31 14:47:00,Bergen Ave & Stegman St,JC108,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.706575,-74.086701,...,14,47,00,2021/07/31,2021/07/31,0 days 00:53:34,3214.0,54.0,0.89,0.04
1,CF7E2039ACB179C2,classic_bike,2021-07-17 19:06:01,2021-07-17 19:20:07,Bergen Ave & Stegman St,JC108,Baldwin at Montgomery,JC020,40.706575,-74.086701,...,19,20,07,2021/07/17,2021/07/17,0 days 00:14:06,846.0,14.0,0.24,0.01
2,9375052379DE50FC,classic_bike,2021-07-08 19:54:34,2021-07-08 20:43:06,Columbus Dr at Exchange Pl,JC106,Liberty Light Rail,JC052,40.716870,-74.032810,...,20,43,06,2021/07/08,2021/07/08,0 days 00:48:32,2912.0,49.0,0.81,0.03
3,EF77D67D538E108E,classic_bike,2021-07-18 20:21:15,2021-07-18 20:28:23,Columbus Dr at Exchange Pl,JC106,Warren St,JC006,40.716870,-74.032810,...,20,28,23,2021/07/18,2021/07/18,0 days 00:07:08,428.0,7.0,0.12,0.00
4,3550C64E6994B8BC,classic_bike,2021-07-08 18:19:07,2021-07-08 18:24:48,Columbus Dr at Exchange Pl,JC106,Warren St,JC006,40.716870,-74.032810,...,18,24,48,2021/07/08,2021/07/08,0 days 00:05:41,341.0,6.0,0.09,0.00


In [34]:
bike_data_df.to_csv("Data/Final_Data/bike_march_dec_2021.csv",index=False)

In [35]:
#create test data set so it does not take a while to load
# this picks random rows from the full data

In [36]:
test_data = bike_data_df.sample(n =round(bike_data_df.shape[0]/10))

In [37]:
test_data.to_csv("Data/Final_Data/test_data.csv",index =False)

In [38]:
bdf = pd.read_csv("Data/Final_Data/bike_march_dec_2021.csv")

In [39]:
bdf.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,End_Hour,End_Minute,End_Second,Start_date,End_date,Ride Duration,Ride Duration Seconds,Ride Duration Mins,Ride Duration Hours,Ride Duration Days
0,2F01C15ECE379AE7,classic_bike,2021-07-31 13:53:26,2021-07-31 14:47:00,Bergen Ave & Stegman St,JC108,South Waterfront Walkway - Sinatra Dr & 1 St,HB103,40.706575,-74.086701,...,14,47,0.0,2021/07/31,2021/07/31,0 days 00:53:34,3214.0,54.0,0.89,0.04
1,CF7E2039ACB179C2,classic_bike,2021-07-17 19:06:01,2021-07-17 19:20:07,Bergen Ave & Stegman St,JC108,Baldwin at Montgomery,JC020,40.706575,-74.086701,...,19,20,7.0,2021/07/17,2021/07/17,0 days 00:14:06,846.0,14.0,0.24,0.01
2,9375052379DE50FC,classic_bike,2021-07-08 19:54:34,2021-07-08 20:43:06,Columbus Dr at Exchange Pl,JC106,Liberty Light Rail,JC052,40.716870,-74.032810,...,20,43,6.0,2021/07/08,2021/07/08,0 days 00:48:32,2912.0,49.0,0.81,0.03
3,EF77D67D538E108E,classic_bike,2021-07-18 20:21:15,2021-07-18 20:28:23,Columbus Dr at Exchange Pl,JC106,Warren St,JC006,40.716870,-74.032810,...,20,28,23.0,2021/07/18,2021/07/18,0 days 00:07:08,428.0,7.0,0.12,0.00
4,3550C64E6994B8BC,classic_bike,2021-07-08 18:19:07,2021-07-08 18:24:48,Columbus Dr at Exchange Pl,JC106,Warren St,JC006,40.716870,-74.032810,...,18,24,48.0,2021/07/08,2021/07/08,0 days 00:05:41,341.0,6.0,0.09,0.00


In [40]:
tdf = pd.read_csv("Data/Final_Data/test_data.csv")

In [41]:
tdf.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,...,End_Hour,End_Minute,End_Second,Start_date,End_date,Ride Duration,Ride Duration Seconds,Ride Duration Mins,Ride Duration Hours,Ride Duration Days
0,C9E8CA08DD1E517D,classic_bike,2021-07-11 02:50:41,2021-07-11 03:55:54,Bloomfield St & 15 St,HB203,Bloomfield St & 15 St,HB203,40.754530,-74.026580,...,3,55,54.0,2021/07/11,2021/07/11,0 days 01:05:13,3913.0,65.0,1.09,0.05
1,9B9095212DFF1EB9,classic_bike,2021-11-09 10:23:58,2021-11-09 10:26:57,Sip Ave,JC056,McGinley Square,JC055,40.730897,-74.063913,...,10,26,57.0,2021/11/09,2021/11/09,0 days 00:02:59,179.0,3.0,0.05,0.00
2,2CE214C0C6C0CADC,classic_bike,2021-09-06 17:36:15,2021-09-06 17:58:50,Hamilton Park,JC009,Hamilton Park,JC009,40.727596,-74.044247,...,17,58,50.0,2021/09/06,2021/09/06,0 days 00:22:35,1355.0,23.0,0.38,0.02
3,946828769516324B,classic_bike,2021-09-20 16:51:00,2021-09-20 16:59:17,6 St & Grand St,HB302,Hoboken Terminal - Hudson St & Hudson Pl,HB101,40.744398,-74.034501,...,16,59,17.0,2021/09/20,2021/09/20,0 days 00:08:17,497.0,8.0,0.14,0.01
4,483BD201ABEAA3A8,classic_bike,2021-11-13 00:28:59,2021-11-13 00:31:02,Newark Ave,JC032,Jersey & 6th St,JC027,40.721525,-74.046305,...,0,31,2.0,2021/11/13,2021/11/13,0 days 00:02:03,123.0,2.0,0.03,0.00


In [42]:
tdf.columns

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual', 'Start_Year', 'Start_Month', 'Start_Day', 'Start_Time',
       'Start_Hour', 'Start_Minute', 'Start_Second', 'End_Year', 'End_Month',
       'End_Day', 'End_Time', 'End_Hour', 'End_Minute', 'End_Second',
       'Start_date', 'End_date', 'Ride Duration', 'Ride Duration Seconds',
       'Ride Duration Mins', 'Ride Duration Hours', 'Ride Duration Days'],
      dtype='object')

In [46]:
tdf["member_casual"].unique()

array(['casual', 'member'], dtype=object)